This is simply a sandbox for exploring data that we have captured from the inmotion robot in the sensory shift experiment.

In [ ]:
%pylab inline

import pandas as pd
import pickle
import json
import seaborn as sns

In [ ]:
SAMPLING_RATE = 200
DT = 1/(SAMPLING_RATE)

In [ ]:
#prefix = "incoming/tim/tim_sensoryshift_06_02.12h54m16"
#prefix = "incoming/neeraj_sensoryshift_06_02.14h08m54"
prefix = "incoming/RS02/RS02_recognitionrecall_08_04.11h10m03"

In [ ]:
dumpf = '%sdump.pickle'%prefix
dump = pickle.load(open(dumpf,'rb'))

In [ ]:
trialsf = '%strials.txt'%prefix
trialdata = pd.read_csv(trialsf,sep=' ')
trialdata.head()

In [ ]:
print("Read %d trials"%len(dump))

In [ ]:
jsonf = '%sparameters.json'%prefix
conf = json.load(open(jsonf,'r'))

In [ ]:
for i,d in enumerate(dump):
    print("trial %d : %d captured samples"%(i,len(d['captured'])))
    #print(len(d['captured']))

In [ ]:
# Convert each trial into data frame
for d in dump:
    rec = pd.DataFrame(d['captured'])
    rec.columns = ['x','y','fx','fy','fz']
    rec['t']=arange(rec.shape[0])/SAMPLING_RATE
    rec['vx']=diff(rec['x'],prepend=[np.nan])/DT
    rec['vy']=diff(rec['y'],prepend=[np.nan])/DT
    rec['vtot']=sqrt(rec['vx']**2 + rec['vy']**2)
    d['traj']=rec

In [ ]:
rec.head()

In [ ]:
from ipywidgets import *

In [ ]:
def plottrial(i):
    f,axs= subplots(2,2,figsize=(12,7))
    ax = axs[0][0]
    trial = dump[i]
    rec = trial['traj']
    ax.plot(rec['x'],rec['y'],'-')    
    sc = conf['review_force_scale']
    for i in range(0,rec.shape[0],10): # subsample a little bit
        row = rec.iloc[i]
        ax.arrow(row['x'],row['y'],sc*row['fx'],sc*row['fy'])
    #ax.plot(trial['target_position'],'o')
    ax.set_xlim(-.3,.3)
    ax.set_ylim(-.1,.2)
    ax.set_aspect('equal')
    
    ax = axs[0][1]
    ax.plot(rec['t'],rec['x'],label='x')
    ax.plot(rec['t'],rec['y'],label='y')

    ax = axs[1][0]
    ax.plot(rec['t'],rec['vx'],label='vx')
    ax.plot(rec['t'],rec['vy'],label='vy')

    ax = axs[1][1]
    ax.plot(rec['t'],rec['vtot'],label='vtot')
    #ax.plot(rec['t'],rec['fy'],label='fy')
    
    
    sns.despine(offset=5)
interact(plottrial,i=(0,len(dump)-1))

# Compute peak velocity

In [ ]:
vmaxx = []
for trial in dump:
    rec = trial['traj']
    vs = [ v for v in rec['vtot'] if ~np.isnan(v) ]
    vmax = max(vs)
    vmaxpoint = rec [ rec['vtot']==vmax ].iloc[0]
    vmaxx.append({'trial':trial['trial'],
                 'vmax':vmax,
                 'vmax_x':vmaxpoint['x'],
                 'vmax_y':vmaxpoint['y']})
vmaxx = pd.DataFrame(vmaxx)

In [ ]:
vmaxx